Clustering (6 Points): Implement k-means and k-mediods algorithms from scratch. To test your algorithm, just use the features from the digits dataset from scikit-learn:

sklearn.datasets.load_digits

The digits dataset consists of 10 classes (digits from 0 to 9). Run your k-means and k-mediods on this dataset with k = 10, and visualize the obtained clustering. Compare the performance of k-means and k-mediods clustering and check if the k-means and k-mediods both reduce the clustering loss.

In [ ]:
from sklearn.datasets import load_digits
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# My K-means implementation
class KMeans(object):

    def __init__(self, n_clusters, n_init, max_iterations):
      """ n_clusters: number of labels the algorithm will split the datasets into
          n_init: number of times the algorithm will run
          max_iterations: max number of iterations the algorithm will run per n_init as long as it does not converge
          """
      self.n_clusters = n_clusters
      self.n_init = n_init
      self.max_iterations = max_iterations

    # conditioning algorithm
    def fit(self, X):
      self.X = X
      self.best_centroids = None
      self.best_inertia = float('inf')
      self.cluster_names = [f"Cluster_{i}" for i in range(self.n_clusters)]
      self.best_df = None
      for i in range(self.n_init):
        centroids = X.sample(self.n_clusters)
        self.inertia = 0
        # rows = rows of X and columns = n_clusters + closest cluster + distance to closest cluster + checkIfChanged
        self.df = pd.DataFrame(np.zeros((X.shape[0], self.n_clusters + 3)), columns=['closest_cluster', 'distance_closest', "changed_class"] + self.cluster_names)

        for j in range(self.max_iterations):
          for i in range(self.n_clusters):
            self.df[f"Cluster_{i}"] = self.euclidian_distance(centroids.iloc[i])

          self.df['changed_class'] = self.df["closest_cluster"] != self.df[self.cluster_names].idxmin(axis=1)
          self.df['closest_cluster'] = self.df[self.cluster_names].idxmin(axis=1)
          self.df['distance_closest'] = self.df[self.cluster_names].min(axis=1)
          self.inertia = self.df['distance_closest'].sum()
          centroids = X.groupby(self.df['closest_cluster']).mean()
          if True in self.df['changed_class']:
            break

        if self.inertia < self.best_inertia:
          self.best_inertia = self.inertia
          self.best_centroids = centroids
          self.best_df = self.df

    # Using euclidian distance to measure distance between data samples
    def euclidian_distance(self,centroid):
      return np.sqrt(((self.X - centroid) ** 2).sum(axis=1))


In [ ]:
# My implementation of K-medoids
class KMedoids(object):
    def __init__(self, n_clusters, n_init, max_iterations):
      """ n_clusters: number of labels the algorithm will split the datasets into
          n_init: number of times the algorithm will run
          max_iterations: max number of iterations the algorithm will run per n_init as long as it does not converge"""
      self.n_clusters = n_clusters
      self.n_init = n_init
      self.max_iterations = max_iterations

    # conditioning on the training set
    def fit(self, X):
      self.X = X
      self.best_centroids = None
      self.best_inertia = float('inf')
      self.cluster_names = [f"Cluster_{i}" for i in range(self.n_clusters)]
      self.best_df = None
      for i in range(self.n_init):
        centroids = X.sample(self.n_clusters)
        self.inertia = 0
        # rows = rows of X and columns = n_clusters + closest cluster + distance to closest cluster + checkIfChanged
        self.df = pd.DataFrame(np.zeros((X.shape[0], self.n_clusters + 3)), columns=['closest_cluster', 'distance_closest', "changed_class"] + self.cluster_names)

        for j in range(self.max_iterations):
          for i in range(self.n_clusters):
            self.df[f"Cluster_{i}"] = self.euclidian_distance(centroids.iloc[i])

          self.df['changed_class'] = self.df["closest_cluster"] != self.df[self.cluster_names].idxmin(axis=1)
          self.df['closest_cluster'] = self.df[self.cluster_names].idxmin(axis=1)
          self.df['distance_closest'] = self.df[self.cluster_names].min(axis=1)
          self.inertia = self.df['distance_closest'].sum()
          centroids = X.groupby(self.df['closest_cluster']).mean()

          for k in range(self.n_clusters):
            centroids.iloc[k] = X.iloc[(X[self.df["closest_cluster"] == f"Cluster_{k}"] - centroids.iloc[k]).abs().sum(axis=1).idxmin()]

          if True in self.df['changed_class']:
            break


        if self.inertia < self.best_inertia:
          self.best_inertia = self.inertia
          self.best_centroids = centroids
          self.best_df = self.df

    # Using euclidian distance to measure distance between data samples
    def euclidian_distance(self,centroid):
      return np.sqrt(((self.X - centroid) ** 2).sum(axis=1))

In [ ]:
# loading the data from sklearn
X, y = load_digits(return_X_y=True, as_frame=True)
X.head()

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_6,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,pixel_7_7
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,9.0,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0


In [ ]:
# K-Means
print("K-Means")
kmeans = KMeans(n_clusters=10, n_init=10, max_iterations=100)
kmeans.fit(X)

# Loss
print("Best Inertia:", kmeans.best_inertia)

# Clusters locations
print("Best Centroids:")
print(kmeans.best_centroids)

K-Means
Best Inertia: 44752.300926785705
Best Centroids:
                 pixel_0_0  pixel_0_1  pixel_0_2  pixel_0_3  pixel_0_4  \
closest_cluster                                                          
Cluster_0              0.0   0.000000   0.284024   6.952663  11.946746   
Cluster_1              0.0   0.196787   6.457831  12.502008  11.859438   
Cluster_2              0.0   0.942857  10.188571  14.440000   7.771429   
Cluster_3              0.0   1.115646  10.040816  13.408163  14.197279   
Cluster_4              0.0   0.122271   4.218341  11.978166  12.222707   
Cluster_5              0.0   0.000000   0.034091   1.931818  11.102273   
Cluster_6              0.0   0.595506   8.758427  14.595506  14.022472   
Cluster_7              0.0   0.000000   1.159341  11.225275   9.532967   
Cluster_8              0.0   0.022346   4.229050  13.139665  11.268156   
Cluster_9              0.0   0.149254   4.686567  12.820896  14.169154   

                 pixel_0_5  pixel_0_6  pixel_0_7  pixe

In [ ]:
# K-Medoids
print("K-Medoids")
kmeans = KMedoids(n_clusters=10, n_init=10, max_iterations=100)
kmeans.fit(X)

# Loss
print("Best Inertia:",kmeans.best_inertia)

# Clusters locations
print("Best Centroids:")
print(kmeans.best_centroids)

K-Medoids
Best Inertia: 52806.274409669175
Best Centroids:
                 pixel_0_0  pixel_0_1  pixel_0_2  pixel_0_3  pixel_0_4  \
closest_cluster                                                          
Cluster_0              0.0        0.0        0.0        1.0       12.0   
Cluster_1              0.0        0.0        9.0       13.0        6.0   
Cluster_2              0.0        0.0        4.0       13.0       16.0   
Cluster_3              0.0        0.0        4.0       15.0       12.0   
Cluster_4              0.0        1.0       12.0       16.0        5.0   
Cluster_5              0.0        0.0        7.0       14.0       15.0   
Cluster_6              0.0        0.0        0.0        8.0       15.0   
Cluster_7              0.0        0.0        1.0       11.0       15.0   
Cluster_8              0.0        0.0        1.0       13.0        7.0   
Cluster_9              0.0        0.0        4.0       14.0       14.0   

                 pixel_0_5  pixel_0_6  pixel_0_7  pi

In my implementations, Kmedoids take a little longer and Kmeans did a better job in reducing the Clustering Loss. Unfortunately, it is not possible to visualize both models' centroids on the data because the data contains 64 dimensions.